В этом задании вам предлагается проанализировать данные одной из американских телекоммуникационных компаний о пользователях, которые потенциально могут уйти.
Измерены следующие признаки:

state — штат США

account_length — длительность использования аккаунта

area_code — деление пользователей на псевдорегионы, использующееся в телекоме

intl_plan — подключена ли у пользователя услуга международного общения

vmail_plan — подключена ли у пользователя услуга голосовых сообщений

vmail_message — количество голосых сообщений, который пользователь отправил / принял

day_calls — сколько пользователь совершил дневных звонков

day_mins — сколько пользователь проговорил минут в течение дня

day_charge — сколько пользователь заплатил за свою дневную активность

eve_calls, eve_mins, eve_charge — аналогичные метрики относительно вечерней активности

night_calls, night_mins, night_charge — аналогичные метрики относительно ночной активности

intl_calls, intl_mins, intl_charge — аналогичные метрики относительно международного общения

custserv_calls — сколько раз пользователь позвонил в службу поддержки

treatment — номер стратегии, которая применялись для удержания абонентов (0, 2 = два разных типа воздействия, 1 = контрольная группа)

mes_estim — оценка интенсивности пользования интернет мессенджерами

churn — результат оттока: перестал ли абонент пользоваться услугами оператора

Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора.

Для этого мы воспользуемся критерием хи-квадрат. Постройте таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn. Для каждой такой таблицы 2x2 применить критерий хи-квадрат можно с помощью функции scipy.stats.chi2_contingency(subtable, correction=False)

In [2]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt
import itertools

In [3]:
data = pd.read_csv("churn_analysis.csv", index_col=False) 
data.head()

,Unnamed: 0,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,0,KS,128,415,no,yes,25,265.1,110,45.07,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,1,OH,107,415,no,yes,26,161.6,123,27.47,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,2,NJ,137,415,no,no,0,243.4,114,41.38,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,3,OH,84,408,yes,no,0,299.4,71,50.90,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,4,OK,75,415,yes,no,0,166.7,113,28.34,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


In [5]:
data.treatment.value_counts()

0    1133
2    1103
1    1097
Name: treatment, dtype: int64

In [6]:
data_1 = data[data['treatment'] == 1]


In [7]:
state = data_1.state.value_counts()

In [8]:
crosstab = pd.pivot_table(data_1, index = 'state', columns = 'churn', aggfunc='size', fill_value=0)
(crosstab)

churn,False.,True.
state,,
AK,19,1
AL,25,5
AR,11,5
AZ,17,2
CA,10,5
CO,17,5
CT,25,5
DC,18,1
DE,16,1


In [19]:
crosstab

churn,False.,True.
state,,
AK,19,1
AL,25,5
AR,11,5
AZ,17,2
CA,10,5
CO,17,5
CT,25,5
DC,18,1
DE,16,1


In [10]:
res = []
for x in (itertools.combinations((crosstab.index), 2)):
    res.append (x)


In [81]:
chi2_contingency_no_cor = []
for x in res[:]:
    chi2_contingency_no_cor.append(sc.stats.chi2_contingency((crosstab.loc[x[0]], crosstab.loc[x[1]]), correction=False)[1])
    #print (sc.stats.chi2_contingency((crosstab.loc[x[0]],crosstab.loc[x[1]]), correction=False)[1])

In [93]:
k = 0
for i in chi2_contingency_no_cor:
    if i<0.05:
        k = k+1
print ('Ответ на первый вопрос:', k)

Ответ на первый вопрос: 34


In [94]:
chi2_contingency_cor = []
for x in res[:]:
    chi2_contingency_cor.append(sc.stats.chi2_contingency((crosstab.loc[x[0]], crosstab.loc[x[1]]), correction=True)[1])

In [96]:
k = 0
for i in chi2_contingency_cor:
    if i<0.05:
        k = k+1
print ('Ответ на второй вопрос:', k)

Ответ на второй вопрос: 0


In [134]:
#scipy.stats.fisher_exact
stat_fisher = []
for x in res[:]:
    stat_fisher.append(sc.stats.fisher_exact(np.array([crosstab.loc[x[0]], crosstab.loc[x[1]]]))[1])
    #print (np.array([crosstab.loc[x[0]], crosstab.loc[x[1]]]))

In [136]:
sc.stats.pearsonr(data['day_calls'], data['mes_estim'])

(-0.051794350587572625, 0.0027798836869756707)

In [137]:
sc.stats.spearmanr(data['day_calls'], data['mes_estim'])

SpearmanrResult(correlation=0.043349880533927444, pvalue=0.012317367189170541)

In [140]:
frame = pd.read_csv("ab_browser_test.csv", sep=",", header=0)
frame.head()
exp_clicks_count = frame[frame["slot"]=="exp"]["n_clicks"].sum()
control_clicks_count = frame[frame["slot"]=="control"]["n_clicks"].sum()
print ("Exp clicks count difference percent: %.3f" % np.round(float(exp_clicks_count)*100./float(control_clicks_count)-100, 3))

Exp clicks count difference percent: 1.614


In [150]:
frame.head()
frame.browser.value_count()

AttributeError: 'Series' object has no attribute 'value_count'

In [145]:
exp_data = frame[frame["slot"]=="exp"]
control_data = frame[frame["slot"]=="control"]
from statsmodels.sandbox.stats.multicomp import multipletests
exp_browser_names = set(exp_data["browser"])
exp_browser_data = {}
for browser_name in exp_browser_names:
    exp_browser_data[browser_name] = exp_data[exp_data["browser"]==browser_name]
control_browser_names = set(control_data["browser"])
control_browser_data = {}
for browser_name in control_browser_names:
    control_browser_data[browser_name] = control_data[control_data["browser"]==browser_name]
all_browser_names = exp_browser_names.union(control_browser_names)
mann_whitney_pvalues = []
for browser_name in all_browser_names:
    exp_b_user_clicks = exp_browser_data[browser_name]["n_clicks"].values
    control_b_user_clicks = control_browser_data[browser_name]["n_clicks"].values
    mann_whitney_stat = sc.stats.mannwhitneyu(control_b_user_clicks, exp_b_user_clicks, alternative="two-sided")
    mann_whitney_pvalues.append(mann_whitney_stat.pvalue)
mann_whitney_pvalues_corrected = multipletests(mann_whitney_pvalues, alpha = 0.05, method = 'holm')
for b in zip(all_browser_names, mann_whitney_pvalues_corrected[0]):
    print ("%s is statistically valuable: %r" % b)
print ("Count of not rejected hypothesis: %i" % len(list(filter(lambda x: not(x), mann_whitney_pvalues_corrected[0]))))


Browser #4 is statistically valuable: False
Browser #14 is statistically valuable: True
Browser #22 is statistically valuable: False
Browser #2 is statistically valuable: False
Browser #17 is statistically valuable: False
Browser #20 is statistically valuable: False
Count of not rejected hypothesis: 5


In [146]:
for browser_name in all_browser_names:
    exp_no_click_count = float(exp_browser_data[browser_name]["n_nonclk_queries"].sum())
    exp_queries_count = float(exp_browser_data[browser_name]["n_queries"].sum())
    control_no_click_count = float(control_browser_data[browser_name]["n_nonclk_queries"].sum())
    control_queries_count = float(control_browser_data[browser_name]["n_queries"].sum())
    exp_percent = exp_no_click_count/exp_queries_count*100
    control_percent = control_no_click_count/control_queries_count*100
    print ("%s no click queries percentage: exp=%f\tcontrol=%f\tdiff=%f" % (browser_name, exp_percent, control_percent, (control_percent-exp_percent)))

Browser #4 no click queries percentage: exp=45.142942	control=46.970930	diff=1.827988
Browser #14 no click queries percentage: exp=43.755617	control=57.590411	diff=13.834794
Browser #22 no click queries percentage: exp=39.853947	control=40.593977	diff=0.740029
Browser #2 no click queries percentage: exp=44.982747	control=45.962747	diff=0.980000
Browser #17 no click queries percentage: exp=36.937413	control=36.299367	diff=-0.638046
Browser #20 no click queries percentage: exp=38.977376	control=40.540485	diff=1.563108
